In [ ]:
!pip install calflops
!pip install torchsummary, torchsummaryX

In [20]:
from calflops import calculate_flops
from transformers import AutoModel, AutoTokenizer

# 模型和分词器
batch_size = 1
max_seq_length = 128
model_save = "/mnt/workspace/.cache/modelscope/langboat/mengzi-bert-base"

# 加载模型
model = AutoModel.from_pretrained(model_save)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_save)

# 文本
text = "这是一段示例文本。"

# 编码文本
inputs = tokenizer(text,
                   add_special_tokens=True,
                   return_attention_mask=True,
                   padding=True,
                   truncation="longest_first",
                   max_length=max_seq_length,
                   return_tensors="pt")  # 返回PyTorch张量

# 计算FLOPs
calculate_flops(model=model,kwargs=inputs,print_results=True)

# print("Bert FLOPs: %s   MACs: %s   Params: %s" % (flops, macs, params))


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  102.27 M
fwd MACs:                                                               8.43 GMACs
fwd FLOPs:                                                              16.88 GFLOPS
fwd+bwd MACs:                                                           25.3 GMACs
fwd+bwd FLOPs:                                                          50.64 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each module cacul

('16.88 GFLOPS', '8.43 GMACs', '102.27 M')

In [32]:
import torch
import os
import random
import os
import numpy as np
import logging
from config import Config
from model import TorchModel, choose_optimizer
from evaluate import Evaluator
from loader import load_data2
#[DEBUG, INFO, WARNING, ERROR, CRITICAL]
logging.basicConfig(level=logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
seed = Config["seed"] 
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
"""
模型训练主程序
"""
config = Config
Config["num_layers"] = 2
Config["model_type"] = "rnn"
#创建保存模型的目录
if not os.path.isdir(config["model_path"]):
    os.mkdir(config["model_path"])
#加载训练数据
train_data = load_data2(config["train_data_path"], config)
#加载模型
model = TorchModel(config)
# 标识是否使用gpu
cuda_flag = torch.cuda.is_available()
if cuda_flag:
    logger.info("gpu可以使用，迁移模型至gpu")
    model = model.cuda()

In [ ]:
from torchsummaryX import summary as summaryX
# 使用torchsummaryX库打印模型概要
# 注意：这里我们使用一个示例输入来调用summaryX
example_input = torch.randint(0, 4623, (100, 100))
summaryX(model, example_input)

In [34]:
model

TorchModel(
  (embedding): Embedding(4623, 256, padding_idx=0)
  (encoder): RNN(256, 256, num_layers=2, batch_first=True)
  (classify): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
!pip install torchsummaryX